This workbook represent an end to end test of the Nillion network. It will test the following:
1. A user (patient) uploading their data in encrypted form and providing access to Monadic
2. Monadic giving compute access to Snipper, a third party
3. Snipper registering a program and running in on the patient's encrypted data

Before running this workbook, ensure Nillion Devnet is running. 

Install all Python prerequisites. 

In [1]:
!pip install nada-dsl==0.5.0
!pip install py-nillion-client==0.5.0
!pip install nillion-python-helpers==0.2.3
!pip install python-dotenv==1.0.0


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Import all necessary libraries

In [32]:
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from dotenv import load_dotenv
from nillion_python_helpers import get_quote_and_pay, create_nillion_client, create_payments_config
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

Load the environment variables from a Devnet .env file

*Please make sure to replace the path with the correct path to your .env file*

In [33]:
import os

home_dir = os.path.expanduser("~")
env_path = os.path.join(home_dir, ".config", "nillion", "nillion-devnet.env")

print(f"Loading environment variables from {env_path}")

load_dotenv(env_path)

for key, value in os.environ.items():
    if key.startswith("NILLION_"):
        print(f"{key}: {value}")

Loading environment variables from /Users/vishakh/.config/nillion/nillion-devnet.env
NILLION_CLUSTER_ID: 9e68173f-9c23-4acc-ba81-4f079b639964
NILLION_BOOTNODE_MULTIADDRESS: /ip4/127.0.0.1/tcp/37939/p2p/12D3KooWMvw1hEqm7EWSDEyqTb6pNetUVkepahKY6hixuAuMZfJS
NILLION_BOOTNODE_WEBSOCKET: /ip4/127.0.0.1/tcp/54936/ws/p2p/12D3KooWMvw1hEqm7EWSDEyqTb6pNetUVkepahKY6hixuAuMZfJS
NILLION_NILCHAIN_CHAIN_ID: nillion-chain-devnet
NILLION_NILCHAIN_JSON_RPC: http://127.0.0.1:48102
NILLION_NILCHAIN_REST_API: http://localhost:26650
NILLION_NILCHAIN_GRPC: localhost:26649
NILLION_NILCHAIN_PRIVATE_KEY_0: 9a975f567428d054f2bf3092812e6c42f901ce07d9711bc77ee2cd81101f42c5
NILLION_NILCHAIN_PRIVATE_KEY_1: 1e491133b9408b39572a29f91644873decea554224b20e2b0b923aeb860a1c18
NILLION_NILCHAIN_PRIVATE_KEY_2: 980488572f235316cdb330191f8bafe4e635efbe88b3a40f5bee9bd21047c059
NILLION_NILCHAIN_PRIVATE_KEY_3: 612bb5173dc60d9e91404fcc0d1f1847fb4459a7d5160d63d84e91aacbf2ab2f
NILLION_NILCHAIN_PRIVATE_KEY_4: 04f5a984eeea9dce4e5e907da

Read in some basic Nillion network information from the environment. 

In [34]:
cluster_id = os.getenv('NILLION_CLUSTER_ID')
chain_id = os.getenv('NILLION_NILCHAIN_CHAIN_ID')
grpc_endpoint = os.getenv('NILLION_NILCHAIN_GRPC')

print(f"Cluster ID: {cluster_id}")
print(f"Chain ID: {chain_id}")
print(f"GRPC Endpoint: {grpc_endpoint}")

Cluster ID: 9e68173f-9c23-4acc-ba81-4f079b639964
Chain ID: nillion-chain-devnet
GRPC Endpoint: localhost:26649


Each actor will need its own node key so we define a utility function here.

In [35]:
import uuid
def gen_node_key():
    return NodeKey.from_seed(uuid.uuid4().hex)

Setup for the Monadic actor

In [36]:
monadic_seed = "monadic_seed"
monadic_userkey = UserKey.from_seed(monadic_seed)
monadic_client = create_nillion_client(monadic_userkey, gen_node_key())
monadic_party_id = monadic_client.party_id
monadic_user_id = monadic_client.user_id

Setup for the Snipper actor

In [37]:
snipper_seed = "snipper_seed"
snipper_userkey = UserKey.from_seed(snipper_seed)
snipper_client = create_nillion_client(snipper_userkey, gen_node_key())
snipper_party_id = snipper_client.party_id
snipper_user_id = snipper_client.user_id

Setup for the Patient actor

In [38]:
patient_seed = "patient_seed"
patient_userkey = UserKey.from_seed(patient_seed)
patient_client = create_nillion_client(patient_userkey, gen_node_key())
patient_party_id = patient_client.party_id
patient_user_id = patient_client.user_id

Payments set up for all network actions

In [39]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

Configuration for the program to use. Change the contents of the below cell to use another program. 

In [40]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"

As a baseline, let's ensure that the user can store the program and run it on their own secret. 

In [41]:
import os

os.environ['RUST_BACKTRACE'] = '1'

print("The patient is storing the program on the network")

async def store_program():
    receipt_store_program = await get_quote_and_pay(
        patient_client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await patient_client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

# Use this in your Jupyter notebook cell
action_id = await store_program()
print(f"Action ID: {action_id}")

print("The patient is storing a secret on the network")

program_id = f"{patient_user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(patient_user_id)

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    patient_client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await patient_client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

print("The patient is running the program on the secret..")

patient_nillion_client_party_id = patient_client.party_id
patient_party_label = "Party1"
party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(patient_party_label, patient_nillion_client_party_id)
compute_bindings.add_output_party(patient_party_label, patient_nillion_client_party_id)
computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    patient_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await patient_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await patient_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

The patient is storing the program on the network
Getting quote for operation...
Quote cost is 2 unil
Submitting payment receipt 2 unil, tx hash C789C877C4EA48CE2DA837C06E2C37CFD024BCB822F0DE2FEC6965D8BF91001C
Action ID: 229XQCZoSQk9zDx4ixKXg8wAqCy7HnmUjcTuMe7D1bUVhNG7kYNF4xeZYjRsEtJUhKFvXwfM65ixFWDRhptfHCqw/double
The patient is storing a secret on the network
Getting quote for operation...
Quote cost is 482 unil
Submitting payment receipt 482 unil, tx hash 7324FFD79EE5E2E7B33106070AB4DA0A70EFBDA4153E992140CC2FB3EC5A140B
The patient is running the program on the secret..
Getting quote for operation...
Quote cost is 3 unil
Submitting payment receipt 3 unil, tx hash E6155BEBF90CEDC0646DE18474DF1D4FA203090A45633A0D0CAA55808CEB6D95


ComputeError: computing: compute unexpected error: unathorized

Have Monadic store the program on the network

TODO: Also have Snipper save the program and replicate the computation

In [11]:
async def store_program():
    receipt_store_program = await get_quote_and_pay(
        monadic_client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await monadic_client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

# Use this in your Jupyter notebook cell
action_id = await store_program()
print(f"Action ID: {action_id}")



Getting quote for operation...
Quote cost is 2 unil
Submitting payment receipt 2 unil, tx hash 794F90BCF977AF347F3DABEF39F4465C81A5FFD31913D4C9EFA4A6F3EA49DD85
Action ID: 3Uaays4LAuJ7QGbAC94etCHAr1dzt9NBnX4Q3QMY16cpJNYstDNBv6efyrgGeKfBAemaECUCxApThCfLxcmJbAPj/double


Have the Patient store a secret on the network such that:
- The Patient has all the regular persmissions
- Monadic is able to modify permissions and compute on the secret
- Snipper is able to compute on the secret

TODO: Establish a chain by having Monadic actually assign compute access to Snipper

In [12]:
program_id = f"{monadic_user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(patient_user_id)
permissions.add_update_permissions(set([monadic_user_id]))
permissions.add_compute_permissions({monadic_user_id: {program_id}})
permissions.add_compute_permissions({snipper_user_id: {program_id}})

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    patient_client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await patient_client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

Getting quote for operation...
Quote cost is 482 unil
Submitting payment receipt 482 unil, tx hash 2F7A286A2CF462EDD950B35CC0F958F94DD00937E878C5277A36E3E3F0986947


Have Monadic run the stored program on the stored secret

TODO: Have Snipper run the stored program on the stored secret

In [14]:
import os

os.environ['RUST_BACKTRACE'] = 'full'

party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, monadic_user_id)
compute_bindings.add_output_party(party_name, monadic_user_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    monadic_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await monadic_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await monadic_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Getting quote for operation...


RuntimeError: requesting price quote: price quote query failed: send failed because receiver is gone